# # Analysis of Hospital Appointment No-Shows
# ---
# * **Project Goal:** To analyze patient and appointment data to identify factors contributing to no-shows.
# * **Data Source:** [Kaggle - Medical Appointment No Shows](https://www.kaggle.com/datasets/joniarroba/noshowappointments)
# * **Steps:**
#   1. Data Loading and Inspection
#   2. Data Cleaning and Preparation
#   3. Exploratory Data Analysis (EDA)

In [10]:
# ## 0. Setup and Imports
# Import the pandas library, which is essential for data manipulation and analysis.
import pandas as pd

# ## 1. Data Loading and Initial Inspection
# First, we load the dataset from the CSV file and perform a quick inspection to understand its structure, data types, and to check for obvious issues like missing values.

In [11]:
# Load the dataset into a pandas DataFrame
# Make sure the .csv file is in the same directory as this notebook
df = pd.read_csv('KaggleV2-May-2016.csv')

# Display the first 5 rows to get a feel for the data
print("--- First 5 Rows of the Dataset ---")
df.head()

--- First 5 Rows of the Dataset ---


,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [12]:
# Get a concise summary of the DataFrame, including data types and non-null counts
print("\n--- Dataset Info ---")
df.info()


--- Dataset Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   PatientId       110527 non-null  float64
 1   AppointmentID   110527 non-null  int64  
 2   Gender          110527 non-null  object 
 3   ScheduledDay    110527 non-null  object 
 4   AppointmentDay  110527 non-null  object 
 5   Age             110527 non-null  int64  
 6   Neighbourhood   110527 non-null  object 
 7   Scholarship     110527 non-null  int64  
 8   Hipertension    110527 non-null  int64  
 9   Diabetes        110527 non-null  int64  
 10  Alcoholism      110527 non-null  int64  
 11  Handcap         110527 non-null  int64  
 12  SMS_received    110527 non-null  int64  
 13  No-show         110527 non-null  object 
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


### Initial Observations from Inspection
# * The dataset has 110,527 entries and 14 columns.
# * There are no missing values, which is great.
# * Some column names have typos or inconsistent casing (e.g., `Hipertension`, `Handcap`, `No-show`).
# * `ScheduledDay` and `AppointmentDay` are currently `object` (text) types and need to be converted to `datetime` for proper analysis.

## 2. Data Cleaning and Preparation
# In this step, we'll clean up the issues identified during inspection. This includes renaming columns, correcting data types, and removing any illogical data points.


In [13]:
# Rename columns for clarity and consistency
df.rename(columns={
    'Hipertension': 'Hypertension',
    'Handcap': 'Handicap',
    'No-show': 'NoShow',
    'PatientId': 'PatientID',
    'AppointmentID': 'AppointmentID'
}, inplace=True)

# Convert date columns from 'object' to 'datetime'
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay'])
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'])

# Remove records with an invalid age (age < 0)
df = df[df['Age'] >= 0]

print("Data cleaning complete. Columns renamed, data types corrected, and invalid rows removed.")


Data cleaning complete. Columns renamed, data types corrected, and invalid rows removed.


# ### Verifying the Cleaned Data
# Let's run `.info()` again to confirm our changes.

In [14]:
# Verify that the data types have been corrected
print("\n--- Cleaned Dataset Info ---")
df.info()


--- Cleaned Dataset Info ---
<class 'pandas.core.frame.DataFrame'>
Index: 110526 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype              
---  ------          --------------   -----              
 0   PatientID       110526 non-null  float64            
 1   AppointmentID   110526 non-null  int64              
 2   Gender          110526 non-null  object             
 3   ScheduledDay    110526 non-null  datetime64[ns, UTC]
 4   AppointmentDay  110526 non-null  datetime64[ns, UTC]
 5   Age             110526 non-null  int64              
 6   Neighbourhood   110526 non-null  object             
 7   Scholarship     110526 non-null  int64              
 8   Hypertension    110526 non-null  int64              
 9   Diabetes        110526 non-null  int64              
 10  Alcoholism      110526 non-null  int64              
 11  Handicap        110526 non-null  int64              
 12  SMS_received    110526 non-null  int64         

## 3. Exploratory Data Analysis (EDA)
# Now that our data is clean, we can start exploring it to find patterns and insights.

# ### 3.1. Numerical Data Summary
# We'll start by computing summary statistics for all numerical columns.

In [15]:
# Generate descriptive statistics for numerical columns
df.describe()

,PatientID,AppointmentID,Age,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMS_received
count,1.105260e+05,1.105260e+05,110526.000000,110526.000000,110526.000000,110526.000000,110526.000000,110526.000000,110526.000000
mean,1.474934e+14,5.675304e+06,37.089219,0.098266,0.197248,0.071865,0.030400,0.022248,0.321029
std,2.560943e+14,7.129544e+04,23.110026,0.297676,0.397923,0.258266,0.171686,0.161543,0.466874
min,3.921784e+04,5.030230e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.172536e+12,5.640285e+06,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.173184e+13,5.680572e+06,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.438963e+13,5.725523e+06,55.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,9.999816e+14,5.790484e+06,115.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000


# #### Observations from Numerical Data
# * **Age:** The average patient age is around 37 years. The range is from 0 to 115, which seems reasonable.
# * **Chronic Conditions:** Since columns like `Hypertension` and `Diabetes` are binary (0 or 1), their `mean` represents the percentage of patients with that condition.
#     * **19.7%** of appointments are for patients with Hypertension.
#     * **7.2%** of appointments are for patients with Diabetes.

 ### 3.2. Categorical Data Summary
# Next, we'll analyze categorical columns to understand the distribution of key variables like the no-show rate and gender.

In [16]:
# Calculate the percentage of no-shows
no_show_percentage = df['NoShow'].value_counts(normalize=True) * 100
print("--- No-Show Percentage ---")
print(no_show_percentage)

--- No-Show Percentage ---
NoShow
No     79.806561
Yes    20.193439
Name: proportion, dtype: float64


In [17]:
# Calculate the gender distribution
gender_distribution = df['Gender'].value_counts()
print("\n--- Gender Distribution ---")
print(gender_distribution)


--- Gender Distribution ---
Gender
F    71839
M    38687
Name: count, dtype: int64


#### Observations from Categorical Data
# * **No-Show Rate:** The overall no-show rate is **20.2%**. This is a key metric for the hospital.
# * **Gender:** The dataset contains significantly more appointments for **females (65%)** than males (35%).

# ### 3.3. Correlation Analysis
# Finally, let's examine the correlation between numerical variables to see how they relate to each other. A correlation matrix is a great tool for this.

In [18]:
# --- 3.3. Correlation Analysis ---

# Select only the columns with numerical data types
numerical_df = df.select_dtypes(include=['number'])

# Now, compute the correlation matrix on the numerical-only DataFrame
correlation_matrix = numerical_df.corr()

print("--- Correlation Matrix ---")
correlation_matrix

--- Correlation Matrix ---


,PatientID,AppointmentID,Age,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMS_received
PatientID,1.000000,0.004023,-0.004121,-0.002877,-0.006436,0.001608,0.011014,-0.007915,-0.009742
AppointmentID,0.004023,1.000000,-0.019106,0.022619,0.012759,0.022632,0.032946,0.014107,-0.256613
Age,-0.004121,-0.019106,1.000000,-0.092463,0.504586,0.292391,0.095810,0.078032,0.012633
Scholarship,-0.002877,0.022619,-0.092463,1.000000,-0.019730,-0.024894,0.035022,-0.008587,0.001192
Hypertension,-0.006436,0.012759,0.504586,-0.019730,1.000000,0.433085,0.087970,0.080083,-0.006270
Diabetes,0.001608,0.022632,0.292391,-0.024894,0.433085,1.000000,0.018473,0.057530,-0.014552
Alcoholism,0.011014,0.032946,0.095810,0.035022,0.087970,0.018473,1.000000,0.004647,-0.026149
Handicap,-0.007915,0.014107,0.078032,-0.008587,0.080083,0.057530,0.004647,1.000000,-0.024162
SMS_received,-0.009742,-0.256613,0.012633,0.001192,-0.006270,-0.014552,-0.026149,-0.024162,1.000000


# Observations from Correlation Matrix
# * The strongest correlation is between **Age** and **Hypertension** (0.5), which is logical. As age increases, the likelihood of having hypertension also increases.
# * Most other variables show very weak correlations with each other. For example, the correlation between receiving an `SMS_received` and any chronic condition is close to zero.
# * `NoShow` is not in the correlation matrix because it is a categorical ('Yes'/'No') variable, not numerical. We will need other techniques to analyze its relationship with other variables.